# Statoil/C-CORE Iceberg Classifier Challenge 

#### Kaggle kernel 필사하기 - 1
Link: <https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d>


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import warnings
warnings.filterwarnings('ignore')
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline
import json

## Load and preview data

이 커널에서는 이미지 데이터를 처음 다뤄보는 beginner를 위한 간단한 튜토리얼을 제공한다. 
<br>

분석에 앞서 iceberg 데이터를 구성하고 있는 변수들에 대한 도메인 지식에 대한 탐구를 선행하여, 데이터에 대한 이해를 돕는 커널이다. 예를 들면 Kaggle에서 제공하고 있는 `band_1`과 `band_2`에 대한 설명이 직관적으로 잘 이해가 되지 않았는데, 이 커널에서는 별도의 자료 조사를 통해 이 부분에 대한 이해를 도와준다.

> WTF is HH HV?
Ok, so this Sentinal Settalite is equivalent to RISTSAT(an Indian remote sensing Sat) and they only Transmit pings in H polarization, AND NOT IN V polarization. Those H-pings gets scattered, objects change their polarization and returns as a mix of H and V. Since Sentinel has only H-transmitter, return signals are of the form of HH and HV only. Don't ask why VV is not given(because Sentinel don't have V-ping transmitter).


In [2]:
# read json file
train = pd.read_json('train.json')
test = pd.read_json('test.json')
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [3]:
print(train.shape)
print(test.shape)

(1604, 5)
(8424, 4)


## Preprocess data
본 커널에서는 band_1, band_2에 해당하는 이미지 데이터를 각각 뽑아서 새로운 이미지 데이터 array를 생성하였다. band_1과 band_2가 본래는 75 x 75 이지만 flatten 처리되어있기 때문에 다시 75 x 75 로 reshaping 해주었다.
<br>

여기서 더 나아가 band_1과 band_2 데이터와 함께, 두 데이터의 픽셀 값을 평균 낸 데이터까지 추가하여 총 3개의 채널로 이루어져 있는 새로운 train dataset을 생성했다. 이 데이터는 X_train 데이터이고, 1604개 이미지, 75 x 75 픽셀, 3개 채널로 이루어져 있다.

In [4]:
X_band_1 = np.array([np.array(band).reshape(75, 75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).reshape(75, 75) for band in train['band_1']])

In [5]:
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis], 
                ((X_band_1 + X_band_2)/2)[:, :, :, np.newaxis]], axis = -1)

In [6]:
print(X_train.shape)

(1604, 75, 75, 3)


## Preview image with Plotly
주어진 iceberg 이미지 데이터가 어떻게 생겼는지 궁금하다면 이 커널처럼 plotly를 이용해 이미지를 보는 방법이 있다. Iceberg 이미지가 3D 형태이기 때문에 본 커널에서는 3D surface plot을 이용해 이미지를 보였다.

In [7]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

def plot3d(c, name):
    # 3D surface plot
    data = [go.Surface(z = c)]
    
    # layout 설정
    layout = go.Layout(title = name,
                      autosize = False,
                      width = 700,
                      height = 700,
                      margin=dict(l=65, r=50, b=65, t=90),
                      )
    
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig)

아래에는 iceberg(is_iceberg = 1)인 경우와 ship(is_iceberg = 0)인 경우를 둘 다 그려보았다. 이 두 이미지를 가지고 일반화를 할 수는 없지만, iceberg인 경우가 elevated 된 지점에서 좀 더 복잡한 형태를 띄고, ship인 경우는 elevated된 지점으 iceberg보다는 평면에 가깝다.

In [8]:
plot3d(X_band_1[12, :, :], 'iceberg')

In [9]:
plot3d(X_band_1[0, :, :], 'ship')

## Simple CNN model
본 커널에서는 간단한 CNN 모델을 소개했다. 그 구조를 하나씩 살펴보고 어떤 기법을 사용했는지 알아보았다.

In [15]:
from sklearn.model_selection import train_test_split

In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

### Model Structure
모델 구조는 다음과 같다. 총 4개의 컨볼루셔널 계층, 2개의 Dense 계층 그리고 마지막에는 sigmoid 함수를 이용해 output을 도출한다.
 + 1st: Conv2D layer - MaxPool layer - Dropout
 + 2nd: Conv2D layer - MaxPool layer - Dropout
 + 3rd: Conv2D layer - MaxPool layer - Dropout
 + 4th: Conv2D layer - MaxPool layer - Dropout
 + 5th: Dense layer 
 + 6th: Dense layer
 + Output layer: sigmoid 함수

In [11]:
def model():
    model = Sequential()
    
    # layer1
    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = (75, 75, 3)))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(Dropout(0.2))
    
    # layer2
    model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(Dropout(0.2))
    
    # layer3
    model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(Dropout(0.2))
    
    # layer4
    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    # Dense1
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    # Dense2
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    # Sigmoid layer(output)
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    
    return model

### Callback
머신러닝/딥러닝에서 모델을 학습할 때, 전체 데이터셋을 모두 다 학습하면 1 epoch이 끝났다고 정의한다. 이 때 너무 많은 epoch는 overfitting을, 너무 적은 epoch는 underfitting을 일으킬 수 있는데 적당한 epoch를 찾는 것은 힘든 일이다.

따라서 딥러닝에서는 callback을 이용한다. Callback에는 크게 **EarlyStopping**과 **ModelCheckpoint**가 있다.
 + EarlyStopping: epoch를 반복할 때, 더 이상 모델이 성능이 어느 정도 이상으로 나아지지 않으면 모델 학습을 중단한다.
  + monitor: 어떤 기준으로 성능을 평가할 것인지를 설정
  + patience: 성능이 증가하지 않는 epoch를 몇 번 허용할 것인지를 설정
  
 + ModelCheckpoint: EarlyStopping에 의해 모델 학습이 중단되었을 때, 중단 전 가장 성능이 좋은 모델의 파라미터를 저장하는 역할을 한다. 주로 validation performance를 기준으로 판단한다.

In [13]:
def get_callbacks(filepath, patience = 2):
    es = EarlyStopping(monitor = 'val_loss', patience = patience, mode = 'min')
    msave = ModelCheckpoint(filepath, save_best_only = True)
    return [es, msave]

filepath = ".model_weights.hdf5"
callbacks = get_callbacks(filepath, patience = 2)

In [16]:
target = train['is_iceberg']

X_train_cv, X_val, y_train_cv, y_val = train_test_split(X_train, target, test_size = 0.2, random_state = 0) 

본격적인 모델 학습에서는 batch size를 30, 최대 epoch을 50회로 설정했는데, 이에 따라 모델 성능이 달라질 수 있으니 적절한 값을 찾는 것이 또다른 중요한 문제라고 보여진다.


최종적으로 이 커널에서 제공한 모델의 성능은 accuracy가 0.7259로 높다고는 볼 수 없었다. 하지만 이 커널은 정말 basic한 모델링 기법을 제공하기 때문에 accuracy가 어느정도 되는지는 중요하지 않다! 다른 커널을 통해 모델을 어떻게 develop할 수 있는지에 대해 탐구해보겠다.

In [17]:
cnn_model = model()
cnn_model.fit(X_train_cv, y_train_cv,
             batch_size = 30,
             epochs = 50,
             verbose = 1,
             validation_data = (X_val, y_val),
             callbacks = callbacks)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       1